<a href="https://colab.research.google.com/github/bartoszrud/free-form-VQA/blob/main/T5_on_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/bartoszrud/free-form-VQA/image-captions/data/prediction_val_sub01/preditctions_gt_q.csv?token=GHSAT0AAAAAABWBBH73NTBU43JNK4SXSTY6YWL4JCA')

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.spatial
import matplotlib.pyplot as plt

from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import pandas as pd
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
import torch
import pandas as pd
from pprint import pprint

import sys
import os
import glob

In [5]:
!pip install --upgrade simplet5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
q_list = "question: " + data['question']                          # questions list to feed the model
n_list = data['answer-gt'] + data["answer-generated"] + " </s>" # answers list to feed the model

dict_data = {'source_text': q_list,
      'target_text': n_list}

df = pd.DataFrame(dict_data)
df.head()

,source_text,target_text
0,question: How many cars are on the crosswalk,0 0 cars are on the crosswalk3 3 cars are on t...
1,question: Are these people wearing helmets,No these people are not wearing helmets0 are t...
2,question: Are the men riding a bicycle,No the men are not riding a bicycleThis the ha...
3,question: How many servings of pizza,8 8 servings of pizza2 2 servings are on the t...
4,question: Is the man barefoot,No the man is not barefootThis the hand of is ...


In [7]:
from simplet5 import SimpleT5

Global seed set to 42


In [8]:
train_data, val_data = train_test_split(df[:-100], test_size=0.2)
test_data = df[-100:]
train_data.shape, val_data.shape, test_data.shape

((17783, 2), (4446, 2), (100, 2))

In [9]:
train_data = train_data.applymap(str)
val_data = val_data.applymap(str)

In [10]:
test_data = test_data.applymap(str)

In [11]:
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df = train_data,
            eval_df = val_data, 
            source_max_token_len=24, 
            target_max_token_len=34, 
            batch_size=8, max_epochs=10, use_gpu=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [13]:
model.load_model("t5","/content/outputs/simplet5-epoch-9-train-loss-0.2571-val-loss-0.4854")

In [14]:
test_data['paraphrased'] = test_data.apply(lambda row : model.predict(row['source_text']), axis=1)
test_data

,source_text,target_text,paraphrased
22229,question: Is the woman focused on her kitchen ...,No the woman is not focused on her kitchen wor...,[Yes the woman is focused on her kitchen workT...
22230,question: What time is displayed on the clock ...,It is 2203515 is is shown on the watch </s>,[It is 1250685 is the train]
22231,question: What color is the dress,The dress is graylong exposure shot of car lig...,[The dress is whiteThe dress is black]
22232,question: Is this person a cook,Yes this person is a cookUphill where are they...,[No this person is not a cookUphill where are ...
22233,question: What country is this taken in,Iran is this taken in1930s around which decade...,[Usa is this taken in1930s around which decade...
...,...,...,...
22324,question: What gender are the children,Male are the children1st is the book </s>,[Male are the children1st is the book]
22325,question: Is the guy in mid air,Yes the guy is in mid airItch the elephant is ...,[No the guy is not in mid airThis the hand of ...
22326,question: What city is this,New york is thisseagulls are flying over the o...,[New york is thisseagulls are flying over the ...
22327,question: How many bolts are holding the sign ...,4 4 bolts are holding the sign onto the buildi...,[0 0 bolts are holding the sign onto the build...


In [20]:
test_data.to_csv(r"/content/outputs/para_phrased.csv", index=False)